<a href="https://colab.research.google.com/github/Page007/rutvikpage/blob/master/Neg_Feels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The initial step for the program is as follows: \\
**Input** : A voter profile **V**, set of candidates **C**, committee size *k*. \\
**Output** : Does there exist at least one committee that satisfies our axiom. 

Overall, the question to answer is that does there exist a voter profile for which there doesn't exist a committee that satisfies  axiom. Thus, we will have proved that our axiom may not be suitable for all cases.

In [ ]:
from itertools import combinations, product
from sys import exit
from math import ceil, floor
def feelingUnion(targetSet, V, description):
  temp = set()
  for i in targetSet:
    temp = temp.union(V[i][description])
  del targetSet
  del description
  del V
  return temp

def feelingIntersection(targetSet, V, description):
  temp = V[targetSet[0]][description]
  for i in targetSet:
    temp = temp.intersection(V[i][description])
  del targetSet
  del description
  del V
  return temp

def isAxiom_1_true(W, V, group):
  # find positive union of voters in our group
  pos_feelings = feelingUnion(group, V, 'approve')
  neg_feelings = feelingIntersection(group, V, 'disapprove')
  
  # checking if the following is true:
  # the committee should contain at least one candidate that at least one voter 
  # in the group likes and it should not contain a candidate that anyone in the
  # group dislikes
  if len(set(W).intersection(pos_feelings)) != 0 and len(set(W).intersection(neg_feelings)) == 0:
    return True
  return False

# OUR ORIGINAL PROPOSED AXIOM IS WTPSC 
# (Weak Trichotomous Proportional Solid Coalition)


def WTPSC(W, V, group, k):
  #print('Checking group : ', group, 'for committee : ', W)
  pos_feelings = feelingUnion(group, V, 'approve')
  neg_feelings = feelingUnion(group, V, 'disapprove')
  # checking if the following is true:
  # the committee should contain at least one candidate that belongs to the 
  # union of all approval sets and does not belong to any voter's disapproval 
  # set
  if len(set(W).intersection(pos_feelings)) < floor(len(group) / ceil(len(V) / k)):
    #print('Not true for:', group, 'with committee : ', W)
    del pos_feelings
    del neg_feelings
    return False
  del pos_feelings
  del neg_feelings
  return True






def WTPSC1(W, V, group, k):
  #print('Checking group : ', group, 'for committee : ', W)
  pos_feelings = feelingUnion(group, V, 'approve')
  neg_feelings = feelingUnion(group, V, 'disapprove')
  # checking if the following is true:
  # the committee should contain at least one candidate that belongs to the 
  # union of all approval sets and does not belong to any voter's disapproval 
  # set
  if len(set(W).intersection(pos_feelings.union(feelingIntersection(group, V, 'indif')))) < floor(len(group) / ceil(len(V) / k)):
    #print('Not true for:', group, 'with committee : ', W)
    del pos_feelings
    del neg_feelings
    return False
  del pos_feelings
  del neg_feelings
  return True

def WTPSC2(W, V, group, k):
  #print('Checking group : ', group, 'for committee : ', W)
  pos_feelings = feelingUnion(group, V, 'approve')
  neg_feelings = feelingUnion(group, V, 'disapprove')
  # checking if the following is true:
  # the committee should contain at least one candidate that belongs to the 
  # union of all approval sets and does not belong to any voter's disapproval 
  # set
  if len(set(W).intersection(pos_feelings.union(feelingUnion(group, V, 'indif')))) < floor(len(group) / ceil(len(V) / k)):
    #print('Not true for:', group, 'with committee : ', W)
    del pos_feelings
    del neg_feelings
    return False
  del pos_feelings
  del neg_feelings
  return True

def findVoterGroup(V, k):
  n = len(V)
  voter_set = set()
  for i in V:
    voter_set.add(i)
  deserving = set()

  # find deserving voter groups of every size
  for iterator in range(ceil(n / k), n + 1):
    deserving = deserving.union(set(combinations(voter_set, iterator)))

  #print('Deserving voter groups: ', deserving)
  eligible = set()
  for i in deserving:
    # check if the groups are cohesive as well
    
    # find union of approval sets of of every voter in the subset 'i'
    pos_union = feelingUnion(i, V, 'approve')
    neg_union = feelingUnion(i, V, 'disapprove')
    
    # if the set difference between positive and negative union of the tuple 'i' 
    # is not null, the subset of candidates is eligible
    if len(pos_union.difference(neg_union)) >= floor(len(V) / ceil(n / k)):
      eligible.add(i)
  del deserving
  return eligible 
def proc(V, C, k):
  flag = False
  # find voter groups V' such that their size is floor(n / k) and \cup A_i^+ \setminus \cup A_i^+ != 0
  eligible = findVoterGroup(V, k)
  x = combinations(C, k)
  #print('Eligible voter groups : ', eligible)
  for W in x:
    # find if there is at least one committee in which at least one voter of the 
    # eligible group gets a candidate in the committee.
    # check if the group gets representation in the committee
    for group in eligible:
      if not WTPSC2(W, V, group, k):
        # this means that some eligible group does not get representation
        # so return FALSE
        break
    else:
      # Axiom is true for every group
      # print('Every group checked for committee ', W,'; WTPSC satisfied..')
      del x
      del eligible
      return True
  else:
    if flag is False:
      #print('No committee satisfies the axiom')
      del x
      del eligible
      return False

The following block can be used for rough test cases:


In [ ]:
# define the candidate set, voter set and the preference method here
C = {1, 2, 3}
voters = {1, 2, 3, 4, 5}

V = {1: {'approve': set(), 'disapprove': set(), 'indif': {1, 2, 3}}, 2: {'approve': set(), 'disapprove': set(), 'indif': {1, 2, 3}}, 3: {'approve': {1}, 'disapprove': {2, 3}, 'indif': set()}, 4: {'approve': {2}, 'disapprove': {1, 3}, 'indif': set()}, 5: {'approve': {3}, 'disapprove': {1, 2}, 'indif': set()}}
proc(V, C, k = 2)

True

The following block generates all the voter profiles and checks for every committee if it follows our axioms

In [ ]:
# generate voter profiles in the following manner:
# find all 3-partitions of the set of candidates to 
# form the space of all weak preferences and then find all the voter profiles 
# possible from the space of all possible weak preferences
def checkProfile(profile):
  # check if any two voters have a non empty intersection
  for i in profile:
    for j in profile:
      if i != j:
        if profile[i]['approve'].intersection(profile[j]['approve']):
          del profile
          return True
  del profile
  return False

def generateVoterProfiles(candidates, voters, k):
  count1 = 0
  flag = False
  possibleWeakOrderings = set(product(range(-1, 2), repeat = len(candidates)))
  #voterPrefSpace = set(combinations(possibleWeakOrderings, len(voters)))
  voterPrefSpace = set(product(possibleWeakOrderings, repeat = len(voters)))
  profile = dict()
  for iterator in voterPrefSpace:
    # iterator always symbolizes profile
    iterator = list(iterator)
    for i in range(len(iterator)):
      flag = True
      profile[i + 1] = dict()
      # now initialize profile[i + 1]['approve'], profile[i + 1]['disapprove'], 
      # profile[i + 1]['indif']
      profile[i + 1]['approve'] = set()
      profile[i + 1]['disapprove'] = set()
      profile[i + 1]['indif'] = set()
      for j in range(len(iterator[i])):
        if iterator[i][j] == 0:
          profile[i + 1]['indif'].add(j + 1)
        elif iterator[i][j] == 1:
          profile[i + 1]['approve'].add(j + 1)
        else:
          profile[i + 1]['disapprove'].add(j + 1)
    del iterator[i]
    if not proc(profile, candidates, k):
      print('error profile: ', profile, k)
      count1 += 1
      exit()
  del profile
  del possibleWeakOrderings
  del voterPrefSpace
  return count1
count1 = 0
'''
candidates = set(range(1, 6))
voters = set(range(1, 6))
for k in range(1, len(candidates) + 1):
  if len(voters) <= k:
    continue; # because this is the trivial case when every voter can get her preference in the committee
  count1 += generateVoterProfiles(candidates, voters, k)
del voters
del candidates
    #print(can, vot, k)
'''
for can in range(4, 5):
  candidates = set(range(1, can + 1))
  for vot in range(4, 5):
    #if len(candidates) in {1, 2, 3, 4} and vot in {1, 2, 4, 3}:
    #  continue
    voters = set(range(1, vot + 1))
    for k in range(1, len(candidates) + 1):
      if len(voters) <= k:
        continue; # because this is the trivial case when every voter can get her preference in the committee
      count1 += generateVoterProfiles(candidates, voters, k)
    del voters
  del candidates
      #print(can, vot, k)

print('New axiom failing count: ', count1)

In [ ]:
from random import randint, seed, choices, choice
from itertools import product, combinations
from math import floor, ceil    
from copy import copy, deepcopy
from sys import exit

In [ ]:
seed(version = 2)



def feelingUnion(targetSet, V, description):
    temp = set()
    for i in targetSet:
        temp = temp.union(V[i][description])
    del targetSet
    del description
    del V
    return temp

def feelingIntersection(targetSet, V, description):
    temp = V[targetSet[0]][description]
    for i in targetSet:
        temp = temp.intersection(V[i][description])
    del targetSet
    del description
    del V
    return temp



# Axiom satisfaction returns YES / NO based on whether the input committee
# and the input profile satisfies our axiom : 


def checkAxiom2(V, k, W):
    n = len(V)
    voter_set = set()
    for i in V:
        voter_set.add(i)
    deserving = set()

  # find deserving voter groups of every size
    for iterator in range(ceil(n / k), n + 1):
        deserving = deserving.union(set(combinations(voter_set, iterator)))

  #print('Deserving voter groups: ', deserving)
    eligible = set()
    for i in deserving:
      # check if the groups are cohesive as well
    
      # find intersection of approval sets of every voter in the subset 'i'
        pos_intersection = feelingIntersection(i, V, 'approve')
        pos_union = feelingUnion(i, V, 'approve')
        neg_union = feelingUnion(i, V, 'disapprove')
    # if the set difference between positive and negative union of the tuple 'i' 
    # is not null, the subset of candidates is eligible
        if len(pos_union.difference(neg_union)) >= ceil(n / k) and len(set(W).intersection(pos_union.union(feelingUnion(i, V, 'indif')))) < floor(len(i) / ceil(len(V) / k)):
            del pos_union
            del neg_union
            del pos_intersection
            del deserving
            return False 
    del pos_union
    del neg_union
    del pos_intersection
    del deserving       
    return True
    #return eligible






def checkAxiom1(V, k, W):
    n = len(V)
    voter_set = set()
    for i in V:
        voter_set.add(i)
    deserving = set()

  # find deserving voter groups of every size
    for iterator in range(ceil(n / k), n + 1):
        deserving = deserving.union(set(combinations(voter_set, iterator)))

  #print('Deserving voter groups: ', deserving)
    eligible = set()
    for i in deserving:
      # check if the groups are cohesive as well
    
      # find intersection of approval sets of every voter in the subset 'i'
        pos_intersection = feelingIntersection(i, V, 'approve')
    
    # if the set difference between positive and negative union of the tuple 'i' 
    # is not null, the subset of candidates is eligible
        if len(pos_intersection) >= ceil(n / k) and len(W.intersection(feelingUnion(i, V, 'approve'))) < floor(len(i) / ceil(n / k)):
            return False        
    del deserving
    return True
    #return eligible







# Voting rules implementations

# The voting rules implemented are:
# 1. sequential PAV
# 2. sequential \alpha-CC


# Define sequential PAV for trichotomous settings.
# Define sequential \alpha-CC for trichotomous settings.

'''
Sequential \alpha-CC for trichotomous settings:
Insert candidates one by one in the committee, for each insertion, find the 
difference between the number of approved and disapproved candidates in the 
committee for every voter and if it is greater than \alpha, give a score of 
1 to the committee, otherwise give it a score of 0.
'''

alpha = 1

def evaluate(voter, committee, profile):
    score = 0
    for i in committee:
        if i in profile[voter]['approve']:
            score += 1
        elif i in profile[voter]['disapprove']:
            score += -1
    if score >= alpha:
        return 1
    return 0


def seqCC(profile, candidateNumber, voterNumber, k):
    W = list()
    av = [i + 1 for i in range(candidateNumber)]
    while len(W) < k:
        maxCandidate = None
        maxScore = -10
        for c in av:
            temp = W.copy()
            temp.append(c)
            score = 0
            for v in range(1, voterNumber + 1):
                score += evaluate(v, temp, profile)
            if score > maxScore:
                maxScore = score 
                maxCandidate = c
        av.remove(maxCandidate)
        W.append(maxCandidate)
    return set(W)

def bordaEval(unsat, profile, c):
    voterScores = dict()
    for i in unsat:
        if c in profile[i]['approve']:
            voterScores[i] = 1
        elif c in profile[i]['disapprove']:
            voterScores[i] = -1
        else:
            voterScores[i] = 0
    ordered = sorted(voterScores.items(), key = lambda kv : (kv[1], kv[0]), reverse = True)[:ceil(len(profile) / k)]
    returnDict = dict()
    returnDict['voters'] = [i[0] for i in ordered]
    returnDict['score'] = sum([i[1] for i in ordered])
    del unsat 
    del profile
    del c
    return returnDict


def SeqMonroeA(profile, candidateNumber, k):
    W = list()
    av = [i + 1 for i in range(candidateNumber)]
    unsat = [i for i in profile]
    while len(W) < k:
        score = dict()
        for c in av:
            # in this bordaEval function, find the top ceil(n / k) voters also
            # and also return the (+1, 0, -1) score by these voters for the
            # candidate c
            score[c] = bordaEval(unsat, profile, c)
        for i in av:
            nextSelect = lambda score, candidateList : max([score[i]['score'] for i in candidateList])
            nsScore = nextSelect(score, av)
            if score[i]['score'] == nsScore:
                nextSelect = i
                av.remove(i)
                W.append(i)
                unsat = list(set(unsat).difference(set(score[nextSelect]['voters'])))
                break
        del score
    return W


# STV implementation



def plurality(temp, av, quota):
    maxScore = -10000
    maxCandidate = list()
    minScore = 10000
    minCandidate = list()
    for i in av:
        score = 0
        for j in temp:
            if i in temp[j]['approve']:
                score += 1
            elif len(temp[j]['approve']) == 0:
                if i in temp[j]['indif'] or len(temp[j]['indif']) == 0:
                    score += 1
        if score > maxScore:
            maxScore = score
            maxCandidate = [i]
        elif score == maxScore:
            maxCandidate.append(i)
        if score < minScore:
            minScore = score
            minCandidate = [i]
        elif score == minScore:
            minCandidate.append(i)
    if maxScore >= quota:
        return (True, maxCandidate)
    return (False, minCandidate)

def removeCandidate(temp, c):
    for i in temp:
        if c in temp[i]['approve']:
            temp[i]['approve'].remove(c)
        elif c in temp[i]['disapprove']:
            temp[i]['disapprove'].remove(c)
        elif c in temp[i]['indif']:
            temp[i]['indif'].remove(c)
    return temp

def removeVoters(temp, c, quota):
    count = 0
    t1 = deepcopy(temp)
    for i in t1:
        if c in temp[i]['approve']:
            temp.pop(i)
            count += 1
        elif (len(temp[i]['approve']) == 0 and (c in temp[i]['indif'])):
            temp.pop(i)
            count += 1
        elif len(temp[i]['approve']) == 0 and len(temp[i]['indif']) == 0:
            temp.pop(i)
            count += 1
        if count == quota:
            del t1
            return

def profileEmpty(temp):
    for i in temp:
        if len(temp[i]['approve']) == 0 and len(temp[i]['disapprove']) == 0 and len(temp[i]['indif']) == 0:
            return True
    return False

def STV(profile, candidateNumber, k):
    quota = floor(len(profile) / (k + 1)) + 1
    temp = deepcopy(profile)
    W = list()
    av = [i + 1 for i in range(candidateNumber)]
    while len(W) < k:
        flag, nextAction = plurality(temp, av, quota)
        if len(nextAction) == 0:
            x = 0
            t1 = list(set([i + 1 for i in range(candidateNumber)]).difference(W))
            while len(W) < k:
                W.append(t1[x])
                x += 1
            del t1
            break
        c = choice(nextAction)
        if flag is True:
            W.append(c)
            # remove candidates that approve c
            removeVoters(temp, c, quota)
        av.remove(c)
        removeCandidate(temp, c)
    return W


count = 0
correct = 0
iterations = int(input('Enter the number of iterations for this round : '))
while count < iterations:
    count += 1
    voterNumber = randint(4, 20)
    candidateNumber = randint(1, 15)
    k = randint(1, candidateNumber)
    # generating |V| number of random numbers between 0 to 3^|C| - 1
    r = choices([i for i in range(0, 3**candidateNumber)], k = voterNumber)
    possibleWeakOrderings = list(product(range(-1, 2), repeat = candidateNumber))
    temp = list()
    for i in range(len(r)):
        temp.append(possibleWeakOrderings[r[i]])
    del r
    del possibleWeakOrderings

    profile = dict()

    # getting profile ready
    for i in range(len(temp)):
        profile[i + 1] = dict()
        profile[i + 1]['approve'] = set()
        profile[i + 1]['disapprove'] = set()
        profile[i + 1]['indif'] = set()
        for j in range(len(temp[i])):
            if temp[i][j] == -1:
                profile[i + 1]['disapprove'].add(j + 1)
            elif temp[i][j] == 1:
                profile[i + 1]['approve'].add(j + 1)
            else:
                profile[i + 1]['indif'].add(j + 1)
    del temp
    #W = seqCC(profile, candidateNumber, voterNumber, k)
    #W = set(SeqMonroeA(profile, candidateNumber, k))
    W = set(STV(profile, candidateNumber, k))
    #if len(W) != k:
    #    for i in profile:
    #        print(i, profile[i])
    #    print(W, k)
    if checkAxiom2(profile, k, W):
        correct += 1
else:
    print('probability : ', correct / iterations)
# profile is ready

Enter the number of iterations for this round : 10000
probability :  0.9999
